### Import Module

In [1]:
from glob import glob
import os, sys
from os.path import join, dirname

import datetime, time
from datetime import datetime
from dateutil.relativedelta import relativedelta

import csv
from glob import glob
import chardet
import numpy as np
import pandas as pd

from tqdm import notebook

### Load Data

In [2]:
root_dir = '../Data'

Collar_dir = root_dir + '/KOR_Collar_Data'
Collar_list = glob(Collar_dir + '/*.CSV')

preprocessed_dir = root_dir + '/Preprocessed(new)'
target_data_list = glob(preprocessed_dir + '/target*.csv')
feature_data_list = glob(preprocessed_dir + '/feature*.csv')

In [3]:
# col_name = ['time', 'runtimeMS', 'voltage', 'accX', 'accY', 'accZ', 'orientation', 'ir', 'full', 'iaq', 'iaqAccuracy', 
#             'rawTemp', 'pressure', 'rawHumidity', 'gasResistance', 'compGasAccuracy', 'gasPercentageAccuracy', 
#             'temperature', 'humidity', 'staticIaq', 'statIaqAccuracy', 'co2Equivalent', 'co2Accuracy', 'breathVocEquivalent',
#             'breathVocAccuracy', 'audioLevel']

In [4]:
for file in Collar_list:
        
    collar_data = pd.read_csv(file)
    collar_data['Realtime'] = ''
    collar_data = collar_data.to_records(index = False)

    temp_file = Collar_dir + '/' + str(file.split('\\')[1])
    print(temp_file)

    with open(temp_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['time', 'runtimeMS', 'voltage', 'accX', 'accY', 'accZ', 'orientation', 'ir', 'full', 'iaq', 'iaqAccuracy', 
        'rawTemp', 'pressure', 'rawHumidity', 'gasResistance', 'compGasAccuracy', 'gasPercentageAccuracy', 
        'temperature', 'humidity', 'staticIaq', 'statIaqAccuracy', 'co2Equivalent', 'co2Accuracy', 'breathVocEquivalent',
        'breathVocAccuracy', 'audioLevel', 'Realtime'])

        for i in range(len(collar_data)):
            # convert unixtimestamp to UTC
            collar_data[i][26] = str(datetime.fromtimestamp(int(collar_data[i][0])) - relativedelta(hours=9))
            writer.writerow(collar_data[i])
    
    f.close()    

../Data/KOR_Collar_Data/KORp10_Collar_data.CSV
../Data/KOR_Collar_Data/KORp11_Collar_data.CSV
../Data/KOR_Collar_Data/KORp12_Collar_data.CSV
../Data/KOR_Collar_Data/KORp13_Collar_data.CSV
../Data/KOR_Collar_Data/KORp14_Collar_data.CSV
../Data/KOR_Collar_Data/KORp15_Collar_data.CSV
../Data/KOR_Collar_Data/KORp17_Collar_data.CSV
../Data/KOR_Collar_Data/KORp5_Collar_data.CSV
../Data/KOR_Collar_Data/KORp6_Collar_data.CSV
../Data/KOR_Collar_Data/KORp7_Collar_data.CSV
../Data/KOR_Collar_Data/KORp8_Collar_data.CSV
../Data/KOR_Collar_Data/KORp9_Collar_data.CSV


In [4]:
def convert24(time_stamp): 
    
    time_stamp = str(pd.to_datetime(time_stamp, infer_datetime_format=True))
    # Checking if last two elements of time 
    # is AM and first two elements are 12 
    
    if time_stamp[-2:] == "AM" or time_stamp[-2:] == "PM":
        if time_stamp[-2:] == "AM" and time_stamp[11:13] == "12": 
            return time_stamp[:11] + "00" + time_stamp[13:-2] 

        # remove the AM  
        elif time_stamp[-2:] == "AM": 
            return time_stamp[:-2] 

        # Checking if last two elements of time 
        # is PM and first two elements are 12 
        elif time_stamp[-2:] == "PM" and time_stamp[11:13] == "12": 
            return time_stamp[:-2] 

        else: 

            # add 12 to hours and remove PM 
            return time_stamp[:11] + str(int(time_stamp[11:13]) + 12) + time_stamp[13:-2]
    
    else: 
        return time_stamp

In [ ]:
collar_col_name = ['time', 'runtimeMS', 'voltage', 'accX', 'accY', 'accZ', 'orientation', 'ir', 'full', 'iaq', 'iaqAccuracy', 
            'rawTemp', 'pressure', 'rawHumidity', 'gasResistance', 'compGasAccuracy', 'gasPercentageAccuracy', 
            'temperature', 'humidity', 'staticIaq', 'statIaqAccuracy', 'co2Equivalent', 'co2Accuracy', 'breathVocEquivalent',
            'breathVocAccuracy', 'audioLevel', 'Realtime']

for i in range(len(Collar_list)):
    print(Collar_list[i])
    collar_data = pd.read_csv(Collar_list[i])           # load ith collar_data
    collar_data = collar_data[collar_col_name]          # 
#     idx = collar_data[collar_data["Realtime"] == "Realtime"].index
#     collar_data.drop(collar_data[collar_data["Realtime"] == "Realtime"].index,inplace=True)
    collar_data = collar_data.to_records(index=False)

    target_data = pd.read_csv(target_data_list[i])
    target_data = target_data.to_records(index=False)
    
    feature_file = preprocessed_dir + '/feature_'+target_data_list[i].split('\\')[1].split('.')[0].split('_')[1] + '.csv'
    print(feature_file)
    
    with open(feature_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['RowID', 'Real_time', 'accX', 'accY', 'accZ', 'orientation', 'ir', 'full', 'iaq', 'iaqAccuracy', 
            'rawTemp', 'pressure', 'rawHumidity', 'gasResistance', 'compGasAccuracy', 'gasPercentageAccuracy', 
            'temperature', 'humidity', 'staticIaq', 'statIaqAccuracy', 'co2Equivalent', 'co2Accuracy', 'breathVocEquivalent',
            'breathVocAccuracy', 'audioLevel'])
        
        start = 0
        
        for row in notebook.tqdm(range(len(target_data))):
            for j in range(start, len(collar_data)):
                if convert24(collar_data[j]["Realtime"]) > convert24(target_data[row]['End Window']):
                    break
                    
                if convert24(collar_data[j]["Realtime"]) < convert24(target_data[row]['Start Window']):
                    start += 1
                    
                if convert24(collar_data[j]["Realtime"]) >= convert24(target_data[row]['Start Window']) and convert24(collar_data[j]["Realtime"]) <= convert24(target_data[row]['End Window']):
                    writer.writerow([target_data[row]['RowID'], collar_data[j]['Realtime'], collar_data[j]['accX'], 
                                     collar_data[j]['accY'], collar_data[j]['accZ'], collar_data[j]['orientation'], 
                                     collar_data[j]['ir'], collar_data[j]['full'], collar_data[j]['iaq'], 
                                     collar_data[j]['iaqAccuracy'], collar_data[j]['rawTemp'], collar_data[j]['pressure'],
                                    collar_data[j]['rawHumidity'], collar_data[j]['gasResistance'], collar_data[j]['compGasAccuracy'], 
                                     collar_data[j]['gasPercentageAccuracy'], collar_data[j]['temperature'], 
                                     collar_data[j]['humidity'], collar_data[j]['staticIaq'], collar_data[j]['statIaqAccuracy'], 
                                     collar_data[j]['co2Equivalent'], collar_data[j]['co2Accuracy'], 
                                     collar_data[j]['breathVocEquivalent'], collar_data[j]['breathVocAccuracy'],
                                    collar_data[j]['audioLevel']])
    f.close()

### Integrate Data

In [4]:
def diff_create(df, accel_k, diff_k):
    df[diff_k]=df[accel_k]
    for i in range(len(df)):
        if i==0:
            continue
        elif df['RowID'][i]!=df['RowID'][i-1]:
            df[diff_k][i]=0
            continue
        else:
            df[diff_k][i]-=df[accel_k][i-1]
    df[diff_k][0]=0

def chord_arc_create (df,diff_x,diff_y,diff_z) :
    df['chord'] = np.sqrt(df[diff_x]**2+df[diff_y]**2+df[diff_z]**2)
    df['arc'] = 2*np.arcsin(df['chord']/2)

In [5]:
preprocessed_data_name = preprocessed_dir + '/preprocessed_data(New collar_2).csv'

with open(preprocessed_data_name, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['RowID', 'Real_time', 'accX', 'accY', 'accZ', 'chord', 'arc', 'orientation', 'orientation_cat', 'ir', 'full',
                     'iaq', 'iaqAccuracy', 'rawTemp', 'pressure', 'rawHumidity', 'gasResistance', 'compGasAccuracy', 
                     'gasPercentageAccuracy', 'temperature', 'humidity', 'staticIaq', 'statIaqAccuracy', 'co2Equivalent', 
                     'co2Accuracy', 'breathVocEquivalent', 'breathVocAccuracy', 'audioLevel', 'sound category', 
                     'Modality_cat', 'Modality'])
    
    for i in range(len(target_data_list)):
        rowid = []

        feature_data = pd.read_csv(feature_data_list[i])
        target_data = pd.read_csv(target_data_list[i])
        target_data = target_data.to_records(index=False)

        diff_create(feature_data,'accX','diff_x')
        diff_create(feature_data,'accY','diff_y')
        diff_create(feature_data,'accZ','diff_z')
        chord_arc_create (feature_data,'diff_x','diff_y','diff_z')

        feature_data = feature_data.to_records(index=False)
        temp = feature_data_list[i].split("\\")[1].split(".")[0].split("_")

        if temp[1][:3] == 'KOR':
            print(temp[1])
            for j in range(len(feature_data)):
                idx = np.where(target_data['RowID'] == feature_data[j]['RowID'])
                new_RowID = '1' + '_' + temp[1][4:] + '_' + str(feature_data[j]['RowID'])
                
                sound_cat = ''
                if feature_data[j]['audioLevel'] == 0:
                    sound_cat = 'Quiet'
                elif feature_data[j]['audioLevel']> 0 and feature_data[j]['audioLevel'] <= 30:
                    sound_cat = 'Moderate'
                else:
                    sound_cat = 'Loud'

                orientation_cat = ''
                if feature_data[j]['orientation'] == 0:
                    orientation_cat = 'Portrait Up Front'
                elif feature_data[j]['orientation'] == 1:
                    orientation_cat = 'Portrait Up Back'
                elif feature_data[j]['orientation'] == 2:
                    orientation_cat = 'Portrait Down Front'
                elif feature_data[j]['orientation'] == 3:
                    orientation_cat = 'Portrait Down Back'
                elif feature_data[j]['orientation'] == 4:
                    orientation_cat = 'Landscape Right Front'
                elif feature_data[j]['orientation'] == 5:
                    orientation_cat = 'Landscape Right Back'
                elif feature_data[j]['orientation'] == 6:
                    orientation_cat = 'Landscape Left Front'
                else:
                    orientation_cat = 'Landscape Left Back'
                
                writer.writerow([new_RowID, feature_data[j]['Real_time'], feature_data[j]['accX'], feature_data[j]['accY'], 
                                 feature_data[j]['accZ'], feature_data[j]['chord'], feature_data[j]['arc'], 
                                 feature_data[j]['orientation'], orientation_cat, feature_data[j]['ir'], feature_data[j]['full'], 
                                 feature_data[j]['iaq'], feature_data[j]['iaqAccuracy'], feature_data[j]['rawTemp'], 
                                 feature_data[j]['pressure'] / 100, feature_data[j]['rawHumidity'], 
                                 feature_data[j]['gasResistance'] / 1000, feature_data[j]['compGasAccuracy'],
                                 feature_data[j]['gasPercentageAccuracy'], feature_data[j]['temperature'], feature_data[j]['humidity'], 
                                 feature_data[j]['staticIaq'], feature_data[j]['statIaqAccuracy'], feature_data[j]['co2Equivalent'], 
                                 feature_data[j]['co2Accuracy'], feature_data[j]['breathVocEquivalent'], 
                                 feature_data[j]['breathVocAccuracy'], feature_data[j]['audioLevel'], sound_cat,
                                 target_data[idx[0][0]]["Modality_cat"], target_data[idx[0][0]]["Modality"]])
        
        else:
            print(temp[1])
            for j in range(len(feature_data)):
                idx = np.where(target_data['RowID'] == feature_data[j]['RowID'])
                new_RowID = '0' + '_' + temp[1][4:] + '_' + str(feature_data[j]['RowID'])
                
                sound_cat = ''
                if feature_data[j]['audioLevel'] == 0:
                    sound_cat = 'Quiet'
                elif feature_data[j]['audioLevel']> 0 and feature_data[j]['audioLevel'] <= 30:
                    sound_cat = 'Moderate'
                else:
                    sound_cat = 'Loud'
                
                orientation_cat = ''
                if feature_data[j]['orientation'] == 0:
                    orientation_cat = 'Portrait Up Front'
                elif feature_data[j]['orientation'] == 1:
                    orientation_cat = 'Portrait Up Back'
                elif feature_data[j]['orientation'] == 2:
                    orientation_cat = 'Portrait Down Front'
                elif feature_data[j]['orientation'] == 3:
                    orientation_cat = 'Portrait Down Back'
                elif feature_data[j]['orientation'] == 4:
                    orientation_cat = 'Landscape Right Front'
                elif feature_data[j]['orientation'] == 5:
                    orientation_cat = 'Landscape Right Back'
                elif feature_data[j]['orientation'] == 6:
                    orientation_cat = 'Landscape Left Front'
                else:
                    orientation_cat = 'Landscape Left Back'
                    
                writer.writerow([new_RowID, feature_data[j]['Real_time'], feature_data[j]['accX'], feature_data[j]['accY'], 
                                 feature_data[j]['accZ'], feature_data[j]['chord'], feature_data[j]['arc'], 
                                 feature_data[j]['orientation'], orientation_cat, feature_data[j]['ir'], feature_data[j]['full'], 
                                 feature_data[j]['iaq'], feature_data[j]['iaqAccuracy'], feature_data[j]['rawTemp'], 
                                 feature_data[j]['pressure'] / 100, feature_data[j]['rawHumidity'], 
                                 feature_data[j]['gasResistance'] / 1000, feature_data[j]['compGasAccuracy'],
                                 feature_data[j]['gasPercentageAccuracy'], feature_data[j]['temperature'], feature_data[j]['humidity'], 
                                 feature_data[j]['staticIaq'], feature_data[j]['statIaqAccuracy'], feature_data[j]['co2Equivalent'], 
                                 feature_data[j]['co2Accuracy'], feature_data[j]['breathVocEquivalent'], 
                                 feature_data[j]['breathVocAccuracy'], feature_data[j]['audioLevel'], sound_cat,
                                 target_data[idx[0][0]]["Modality_cat"], target_data[idx[0][0]]["Modality"]])            

<ipython-input-4-4c135b961834>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[diff_k][i]-=df[accel_k][i-1]
<ipython-input-4-4c135b961834>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[diff_k][i]=0
<ipython-input-4-4c135b961834>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[diff_k][0]=0
C:\Users\sckim\.conda\envs\grad\lib\site-packages\pandas\core\series.py:726: RuntimeWarning: invalid value encountered in arcsin
  re

KORp10
KORp11
KORp12
KORp13
KORp14
KORp15
KORp17
KORp5
KORp6
KORp7
KORp8
KORp9
